In [16]:
import requests
from google.cloud import bigquery

In [17]:
import requests
from google.cloud import bigquery

# Function to fetch report data from Workday API
def fetch_workday_report(url, username, password):
    # Define HTTP basic authentication credentials
    auth = (username, password)
    
    # Make HTTP request to Workday API and fetch report data
    response = requests.get(url, auth=auth)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        try:
            # Attempt to parse the response content as JSON
            report_data = response.json()
            return report_data
        except ValueError as e:
            # Print the response content if it's not valid JSON
            print('Response content:', response.text)
            print('Error parsing JSON:', e)
            return None
    else:
        # Print an error message if the request was not successful
        print('Failed to fetch data:', response.status_code)
        return None

# Function to load data into BigQuery
def load_data_into_bigquery(data, project_id):
    client = bigquery.Client(project=project_id)
    dataset_id = 'people_derived_tables'  # Define your dataset ID here
    table_id = 'people_events'  # Define your table ID here
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    
    job_config = bigquery.LoadJobConfig(schema=schema)
    job = client.load_table_from_json(data, table_ref, job_config=job_config)
    job.result()  # Wait for the job to complete
    print(f'Data loaded into BigQuery table {dataset_id}.{table_id}')

# Main function to orchestrate the process
def main():
    url = 'https://services1.myworkday.com/ccx/service/customreport2/smartcontract/joe.daly%40smartcontract.com/PA__Events?Business_Processes%21WID=c24592468ed147b2ac6d0de4d699a7da&Transaction_Status%21WID=1b0f3f40cfaa48dd9f1c86a154a1232a!d598e64d72444e0ea1d39bb88a52b601!b90bc51be01d4ae99b603b02b073714d&Event_Effective_Date_On_or_After=2020-01-01-08%3A00&format=json'
    
    # Enter your Workday API credentials here
    username = 'Workday_Fivetran'
    password = 'jrz!vet2khz1fxe!KCJ'
    
    report_data = fetch_workday_report(url, username, password)
    if report_data:
        project_id = 'ppl-stage-bigquery'
        load_data_into_bigquery(report_data, project_id)

if __name__ == '__main__':
    main()




DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.